In [51]:
import tensorflow as tf
import keras
from keras import layers

In [52]:
data_path = r'C:\Users\narut\OneDrive - University of Hertfordshire\CNN-Data-Science-Project\RealWaste'
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_path,
    image_size=(224, 224),
    batch_size=32,

)


Found 4752 files belonging to 9 classes.


In [53]:
def normalize_img(image, label):
    image = image / 255.0  # Scale pixel values to [0, 1]
    return image, label

dataset = dataset.map(normalize_img)


In [54]:
for images, labels in dataset.take(1):  # Take a single batch
    print(f"Min pixel value: {tf.reduce_min(images).numpy()}")
    print(f"Max pixel value: {tf.reduce_max(images).numpy()}")

Min pixel value: 0.0
Max pixel value: 1.0
